<a href="https://colab.research.google.com/github/rkap786/reading_idm/blob/main/BERTembedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ryxi3uav
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ryxi3uav
  Resolved https://github.com/huggingface/transformers.git to commit 8de7b1ba8d126a6fc9f9bcc3173a71b46f0c3601
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10366449 sha256=38ff24b2709e732222459dd33f447b01dcb45231df93f5568514f210f4029a31
  Stored in directory: /tmp/pip-ephem-wheel-cache-gfm2e0vp/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [2]:
import transformers
import torch
from transformers import AutoTokenizer, ModernBertForMaskedLM, AutoModelForMaskedLM


#*** pretrained_model_vectorizer.py
# Library which includes all code that involves the use of pretrained
# models for contextualization
# Functions:
#    vectorize_with_pretrained_embeddings(sentences, labels): Given a list of text examples,
#    produces embeddings of dim 768 for each example
#***


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import pandas as pd
f= pd.read_csv("/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Data/Processed/data_passage_ques_options_pv.csv")

In [ ]:
#f.head()
#f['Passage'][1]

'"No one but a man can do this," the business manager of the World, a New York newspaper, said to the young woman. The year was 1888. A popular book at the time told about a character who traveled around the world in 80 days. Now Nellie Bly, a young reporter for the newspaper, wanted to do it in real life.\n "Very well,\' said Nellie. "Start the man, and I\'ll start the same day for some other newspaper and beat him."\n In those days it was very unusual for a woman to travel alone. But Nellie Bly was an unusual woman. Her real name was Elizabeth Jane Cochran. When she was 20, she wrote a fiery letter to the editor of the Pittsburgh Dispatch. The editor was so impressed with her letter that he offered her a job as a writer.\n It wasn\'t considered "proper" to use a woman\'s name in a newspaper. So the editor signed Elizabeth\'s work Nellie Bly, a name from a popular song.\n Back then, women were only supposed to write about things considered to be "women\'s topics," such as fashion and 

In [5]:
question= f['QuestionText']
option1= f['option_correct_ans']
option2= f['question_dis1']
option3= f['question_dis2']
option4= f['question_dis3']
# Question + Correct answer
ques_correct_answer= f['question_correct_ans']
ques_distractor1= f['option_distractor1']
ques_distractor2= f['option_distractor2']
ques_distractor3= f['option_distractor3']
# Tag + Question + Correct answer
ques_correct_answer_tag= f['QuestionText'] + '\n Correct answer: ' + f['option_correct_ans']
ques_distractor1_tag=  f['QuestionText'] + '\n Wrong answer 1: ' + f['option_distractor1']
ques_distractor2_tag= f['QuestionText'] + '\n Wrong answer 2: ' + f['option_distractor2']
ques_distractor3_tag= f['QuestionText'] + '\n Wrong answer 3: ' + f['option_distractor3']

# Passage +
ques_alloptions_tag = 'Question: ' + f['QuestionText'] +  '\n Correct answer: ' + f['option_correct_ans'] +  '\n Wrong answer 1: ' + f['option_distractor1'] +  '\n Wrong answer 2: ' + f['option_distractor2'] +  '\n Wrong answer 3: ' + f['option_distractor3'] + "\n Passage:" + f['Passage']




In [ ]:
ques_alloptions_tag[1]

'Question:  Read this sentence from paragraph 6.\n Traveling east across the Atlantic, Nellie took just one bag in order to move\n quickly. \n How is this detail important to paragraph 1?\n Correct answer: It shows one way to help her reach her goal.\n Wrong answer 1: It shows how she is the same as the character in the book.\n Wrong answer 2: It shows a young reporter exploring the world in real life.\n Wrong answer 3: It shows that a young woman taking a trip alone is unusual.\n Passage:"No one but a man can do this," the business manager of the World, a New York newspaper, said to the young woman. The year was 1888. A popular book at the time told about a character who traveled around the world in 80 days. Now Nellie Bly, a young reporter for the newspaper, wanted to do it in real life.\n "Very well,\' said Nellie. "Start the man, and I\'ll start the same day for some other newspaper and beat him."\n In those days it was very unusual for a woman to travel alone. But Nellie Bly was a

In [ ]:
# print(ques_distractor1_tag[1])
# print(ques_distractor1[1])
# print(f['QuestionText'][1])
# print("Everything in one \n" + ques_alloptions_tag[1])


 Read this sentence from paragraph 6.
 Traveling east across the Atlantic, Nellie took just one bag in order to move
 quickly. 
 How is this detail important to paragraph 1?
 Wrong answer 1: It shows how she is the same as the character in the book.
It shows how she is the same as the character in the book.
 Read this sentence from paragraph 6.
 Traveling east across the Atlantic, Nellie took just one bag in order to move
 quickly. 
 How is this detail important to paragraph 1?
Everything in one 
 Read this sentence from paragraph 6.
 Traveling east across the Atlantic, Nellie took just one bag in order to move
 quickly. 
 How is this detail important to paragraph 1?
 Correct answer: It shows one way to help her reach her goal.

 Wrong answer 1: It shows how she is the same as the character in the book.

 Wrong answer 2: It shows a young reporter exploring the world in real life.

 Wrong answer 3: It shows that a young woman taking a trip alone is unusual.
"No one but a man can do this

In [ ]:
f.shape

(1087, 18)

In [9]:
def vectorize_with_pretrained_embeddings(sentences):
  """
  Produces a tensor containing a BERT embedding for each sentence in the dataset or in a
  batch
  Args:
    sentences: List of sentences of length n (max length 8192)
  Returns:
    embeddings: A 2D torch array containing embeddings for each of the n sentences (n x d)
                where d = 50368
  """

  tokenizer = AutoTokenizer.from_pretrained('answerdotai/ModernBERT-base')
  pretrained_model = AutoModelForMaskedLM.from_pretrained('answerdotai/ModernBERT-base', output_hidden_states=False)
  pretrained_model.eval()
  embeddings = []
  for sentence in sentences:
    #with_tags = "[CLS] " + sentence + " [SEP]"
    tokenized_sentence= tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    # print(tokenizer.decode(tokenized_sentence['input_ids'][0]))
    # print(tokenized_sentence['input_ids'][0])
    # print(len(tokenized_sentence['input_ids'][0]))

    # indices_from_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    # segments_ids = [1] * len(indices_from_tokens)
    # tokens_tensor = torch.tensor([indices_from_tokens])
    # segments_tensors = torch.tensor([segments_ids])
    # print("indices from token",indices_from_tokens)
    #print(tokens_tensor)
    # print(segments_tensors)
    with torch.no_grad():
      outputs = pretrained_model(**tokenized_sentence)[0] # The output is the
      #last hidden state of the pretrained model of shape 1 x sentence_length x BERT embedding_length
      #print('outputs: ', outputs)
      embeddings.append(torch.mean(outputs, dim = 1))# we average across the embedding length
      #dimension to produce constant sized tensors
  print(embeddings[0].shape)
  embeddings = torch.cat(embeddings, dim = 0)
  print('Shape of embeddings tensor (n x d = 768): ', embeddings.shape)
  return embeddings.cpu().detach().numpy()

In [ ]:
# ## new code
# input = ["The boy is running", "Cat is dog"]
# print(input)
# embedding = vectorize_with_pretrained_embeddings_new2(input)
# #print(embedding.shape)

['The boy is running']


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[CLS]The boy is running[SEP]
tensor([50281,   510,  5006,   310,  3515, 50282])
6
indices from token tensor([50281,   510,  5006,   310,  3515, 50282])
outputs:  tensor([[[ 6.8395, -1.2113,  7.5360,  ..., -1.4441, -1.4544, -1.4158],
         [ 2.8965,  0.8987, 11.3284,  ..., -1.9701, -1.9811, -1.9754],
         [-6.4731, -3.9625,  3.0849,  ..., -3.3811, -3.3826, -3.3737],
         [-7.5385, -3.6553,  7.2048,  ..., -3.9507, -3.9301, -3.9403],
         [-4.4744, -2.9361, 11.6298,  ..., -2.9843, -2.9808, -2.9799],
         [ 6.3464, -3.5881,  6.8937,  ..., -2.1259, -2.1302, -2.1258]]])
torch.Size([1, 50368])
Shape of embeddings tensor (n x d = 768):  torch.Size([1, 50368])


In [ ]:
#Test sentences. To test, run 'python pretrained_model_vectorizer.py' at the command line
#sentences = ["The boy is running", "The dog has been barking for the whole evening"]
#To use with real data, import pretrained_model_vectorizer, then call
#'vectorize_with_pretrained_embeddings' on your list of sentences to embed
#question.embed= vectorize_with_pretrained_embeddings(question)

In [ ]:
# quesoptionembed_corr= vectorize_with_pretrained_embeddings(ques_correct_answer)
# quesoptionembed1= vectorize_with_pretrained_embeddings(ques_distractor1)
# quesoptionembed2= vectorize_with_pretrained_embeddings(ques_distractor2)
# quesoptionembed3= vectorize_with_pretrained_embeddings(ques_distractor3)

In [ ]:
# quesembed= vectorize_with_pretrained_embeddings(question)
# corrembed= vectorize_with_pretrained_embeddings(option1)
# optionembed1= vectorize_with_pretrained_embeddings(option2)
# optionembed2= vectorize_with_pretrained_embeddings(option3)
# optionembed3= vectorize_with_pretrained_embeddings(option4)

In [ ]:
# corrembedtag= vectorize_with_pretrained_embeddings(ques_correct_answer_tag)
# optionembed1tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)
# optionembed2tag= vectorize_with_pretrained_embeddings(ques_distractor2_tag)
# optionembed3tag= vectorize_with_pretrained_embeddings(ques_distractor1_tag)

In [10]:
ques_alldistractors= vectorize_with_pretrained_embeddings(ques_alloptions_tag)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Streaming output truncated to the last 5000 lines.
         [ -2.9870,  -2.3686,  15.1163,  ...,  -2.9075,  -2.9158,  -2.9058],
         [  5.9804,  -3.9452,   5.9795,  ...,  -2.3826,  -2.3848,  -2.3864]]])
outputs:  tensor([[[ -5.4271,  -2.8616,  13.3633,  ...,  -2.6706,  -2.6544,  -2.6354],
         [  5.8868,  -0.6881,   4.4137,  ...,  -1.5477,  -1.5464,  -1.5274],
         [ -0.9852,  -4.6376,   8.1237,  ...,  -3.9878,  -3.9798,  -3.9722],
         ...,
         [-13.5502,  -7.0293,  16.3273,  ...,  -5.9834,  -5.9481,  -5.9530],
         [ -2.8591,  -2.3657,  15.4242,  ...,  -2.9241,  -2.9316,  -2.9218],
         [  5.9607,  -3.9272,   5.9866,  ...,  -2.3690,  -2.3714,  -2.3726]]])
outputs:  tensor([[[ -6.7365,  -3.2788,  12.9816,  ...,  -2.9472,  -2.9282,  -2.9111],
         [  5.9646,  -0.4906,   3.8147,  ...,  -1.3223,  -1.3228,  -1.3014],
         [ -1.5493,  -4.4929,   8.3853,  ...,  -3.7513,  -3.7495,  -3.7391],
         ...,
         [-12.7681,  -6.6325,  15.5019,  ...,  -5.

In [11]:
import numpy as np
from numpy import savetxt
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df= pd.DataFrame(ques_alldistractors)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_alldistractors_embed_mbert.csv')


/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings


In [ ]:
import numpy as np
from numpy import savetxt
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'
df= pd.DataFrame(quesoptionembed_corr)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_embed_correct.csv')

df= pd.DataFrame(quesoptionembed1)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_embed_dis1.csv')

df= pd.DataFrame(quesoptionembed2)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_embed_dis2.csv')

df= pd.DataFrame(quesoptionembed3)
df['PassNumUnq'] = f['PassNumUnq']
df['QNoUnq'] = f['QNoUnq']
df.to_csv('question_embed_dis3.csv')



#df= pd.DataFrame(quesembed)
#df.to_csv('question_embed.csv')

#df= pd.DataFrame(corrembed)
#df.to_csv('option_embed_correct.csv')

#df= pd.DataFrame(optionembed1)
#df.to_csv('optionembed1.csv')

#df= pd.DataFrame(optionembed2)
#f.to_csv('optionembed2.csv')

#df= pd.DataFrame(optionembed3)
#df.to_csv('optionembed3.csv')

#np.savetxt('/Users/radhika/Downloads/question_embed.csv', question.embed, delimiter=',')
#np.save(question.embed,)


/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings


NameError: name 'quesoptionembed_corr' is not defined

In [ ]:
import numpy as np
from numpy import savetxt
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/Embeddings/'

df= pd.DataFrame(corrembedtag)
df.to_csv('option_embed_correct_tag.csv')

df= pd.DataFrame(optionembed1tag)
df.to_csv('optionembed1_tag.csv')

df= pd.DataFrame(optionembed2tag)
df.to_csv('optionembed2_tag.csv')

df= pd.DataFrame(optionembed3tag)
df.to_csv('optionembed3_tag.csv')

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.330693,0.038969,0.069046,0.141347,0.562531,-0.034794,-0.045604,0.071342,-0.053162,-0.217772,...,0.317475,0.156087,-0.123962,-0.269952,0.094822,0.145907,-0.207998,0.324924,0.330233,-0.009789
1,0.150200,0.274270,-0.039747,0.272531,0.666258,-0.126865,0.024354,-0.013578,-0.016012,-0.114078,...,0.208553,0.335136,-0.002739,-0.248087,0.044126,0.209430,-0.082822,0.187699,0.172975,0.142289
2,0.316193,-0.114018,-0.007403,0.150899,0.482411,-0.054715,0.255445,0.073818,-0.062459,-0.065642,...,0.376271,0.203655,-0.218886,-0.109045,0.187939,0.178523,-0.090680,0.405097,0.209188,-0.032180
3,0.451735,-0.035361,0.060584,0.046701,0.494822,0.024858,0.058867,0.131332,-0.004860,-0.290856,...,0.245316,0.166824,-0.153981,-0.259816,-0.038695,0.186564,-0.191631,0.309455,0.253131,0.001289
4,0.375125,-0.001411,0.016060,-0.089303,0.024385,0.141236,0.317772,0.116051,-0.206724,-0.230045,...,0.256846,0.177193,-0.099984,-0.219399,0.038975,0.209707,-0.124114,0.087240,0.294826,0.216154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,0.374191,-0.092379,0.249997,0.309695,0.122369,-0.153756,0.123190,0.003419,0.053187,0.195876,...,-0.048363,0.057822,-0.082947,-0.117225,0.034189,-0.216345,-0.063492,0.308002,0.208320,-0.122181
1112,0.197396,-0.015915,0.096769,0.203303,0.251318,-0.227677,0.172645,-0.086674,-0.062594,-0.002558,...,0.036880,0.011768,0.054642,0.074262,0.104311,-0.230042,-0.028243,0.311628,-0.011404,-0.008374
1113,0.297722,-0.003906,0.007676,0.123774,0.052911,-0.238047,0.119631,0.062502,0.060466,0.146408,...,0.037549,0.198642,-0.075447,-0.001407,0.087634,-0.141926,-0.089317,0.286568,0.183502,-0.043947
1114,0.408570,0.140626,0.007848,0.179747,0.132918,-0.180473,0.132025,0.076565,-0.043467,0.182478,...,0.205288,0.061859,-0.020326,-0.103721,0.089978,-0.167842,-0.041928,0.132927,0.166269,-0.081589


In [ ]:
df

(1116, 768)

In [ ]:
question.embed

array([[ 0.33069327,  0.0389686 ,  0.06904568, ...,  0.3249241 ,
         0.33023322, -0.00978899],
       [ 0.15020001,  0.27427033, -0.03974681, ...,  0.18769884,
         0.17297511,  0.14228924],
       [ 0.31619295, -0.11401785, -0.00740315, ...,  0.40509737,
         0.20918822, -0.03217997],
       ...,
       [ 0.29772183, -0.00390649,  0.00767591, ...,  0.28656772,
         0.18350151, -0.04394712],
       [ 0.4085702 ,  0.14062564,  0.00784751, ...,  0.13292709,
         0.1662686 , -0.0815894 ],
       [ 0.359874  ,  0.0653155 , -0.03084283, ...,  0.26369065,
         0.03838476, -0.06845871]], dtype=float32)